# 10장. 평가

In [ ]:
!pip install langchain langchain-community langchain-openai langgraph langsmith

In [ ]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')

---
## 코드 10-1~10-2 검색 및 관련성 평가 (Retrieve & Grade)

# 검색 및 관련성 평가 (Retrieve & Grade)

이 노트북에서는 **RAG 시스템에서 검색 결과의 관련성을 평가**하는 방법을 배웁니다.

## RAG 평가의 중요성

```
┌────────────────────────────────────────────────────────────────────┐
│                    RAG 평가가 필요한 이유                           │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   RAG 시스템:                                                      │
│   질문 → 검색 → 문서들 → LLM → 답변                               │
│                                                                    │
│   문제:                                                            │
│   검색된 문서가 질문과 관련 없으면?                                │
│   → LLM이 잘못된 정보로 답변                                       │
│   → 환각(Hallucination) 발생                                       │
│                                                                    │
│   해결:                                                            │
│   검색 결과의 관련성을 LLM으로 평가!                               │
│   → 관련 없는 문서 필터링                                          │
│   → 필요시 재검색 또는 웹 검색                                     │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 아키텍처

```
┌────────────────────────────────────────────────────────────────────┐
│                    검색 + 평가 흐름                                 │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   질문: "2024년 LangGraph 에이전트 사례는?"                        │
│     │                                                              │
│     ▼                                                              │
│   ┌──────────────┐                                                 │
│   │   Retriever  │  → 문서 4개 검색                                │
│   └──────┬───────┘                                                 │
│          │                                                         │
│          ▼                                                         │
│   ┌──────────────┐                                                 │
│   │    Grader    │  → 각 문서 관련성 평가 (yes/no)                │
│   │ (LLM 평가기) │                                                 │
│   └──────┬───────┘                                                 │
│          │                                                         │
│   관련 문서만 선택                                                  │
│          │                                                         │
│          ▼                                                         │
│   ┌──────────────┐                                                 │
│   │   Generator  │  → 관련 문서로 답변 생성                        │
│   └──────────────┘                                                 │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2
!ollama pull nomic-embed-text

# 2. 문서 로드 및 인덱싱

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import InMemoryVectorStore
from langchain_ollama import OllamaEmbeddings

# 블로그 게시물 URL
urls = [
    "https://blog.langchain.dev/top-5-langgraph-agents-in-production-2024/",
    "https://blog.langchain.dev/langchain-state-of-ai-2024/",
    "https://blog.langchain.dev/introducing-ambient-agents/",
]

# 문서 로드
print("=== 문서 로드 중 ===")
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
print(f"로드된 문서: {len(docs_list)}개")

In [ ]:
# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, 
    chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)
print(f"분할된 청크: {len(doc_splits)}개")

In [ ]:
# 벡터 스토어 생성
embeddings = OllamaEmbeddings(model='nomic-embed-text')

vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits,
    embedding=embeddings,
)
retriever = vectorstore.as_retriever()

print("✅ 벡터 스토어 및 Retriever 생성 완료")

# 3. 검색 테스트

In [ ]:
# 검색 테스트
question = "2024년에 프로덕션 환경에서 사용된 LangGraph 에이전트 2개는 무엇인가요?"

results = retriever.invoke(question)

print(f"=== 검색 결과 ({len(results)}개) ===")
for i, doc in enumerate(results):
    print(f"\n[문서 {i+1}]")
    print(f"내용: {doc.page_content[:200]}...")

# 4. 관련성 평가 스키마 정의

In [ ]:
from pydantic import BaseModel, Field

class GradeDocuments(BaseModel):
    """검색된 문서의 관련성 평가 결과"""
    
    binary_score: str = Field(
        description="문서가 질문과 관련이 있으면 'yes', 없으면 'no'"
    )

print("✅ GradeDocuments 스키마 정의 완료")

# 5. 관련성 평가기 (Grader) 생성

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# LLM 설정 (Structured Output)
llm = ChatOllama(model='llama3.2', temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# 평가 프롬프트
system = """당신은 사용자 질문에 대한 검색된 문서의 관련성을 평가하는 채점자입니다.
문서에 질문과 관련된 키워드나 의미가 포함되어 있다면 관련성이 있다고 평가하세요.
문서가 질문과 관련이 있는지 여부를 나타내는 'yes' 또는 'no'로 평가해 주세요."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "검색된 문서: \n\n {document} \n\n 사용자 질문: {question}"),
    ]
)

# 평가 체인
retrieval_grader = grade_prompt | structured_llm_grader

print("✅ 관련성 평가기 생성 완료")

# 6. 검색 결과 평가

In [ ]:
# 각 문서 평가
print("=== 관련성 평가 ===")
print(f"질문: {question}\n")

relevant_docs = []

for i, doc in enumerate(results):
    result = retrieval_grader.invoke({
        "question": question, 
        "document": doc.page_content
    })
    
    status = "✅ 관련" if result.binary_score == 'yes' else "❌ 무관"
    print(f"[문서 {i+1}] {status}")
    print(f"  내용: {doc.page_content[:100]}...")
    print()
    
    if result.binary_score == 'yes':
        relevant_docs.append(doc)

print(f"\n관련 문서: {len(relevant_docs)}개 / 전체: {len(results)}개")

# 7. 관련 문서로 답변 생성

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 답변 생성 프롬프트
answer_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 질문에 답변하는 어시스턴트입니다. 주어진 문서를 바탕으로 답변하세요."),
    ("human", "문서:\n{context}\n\n질문: {question}")
])

# 컨텍스트 생성
context = "\n\n".join([doc.page_content for doc in relevant_docs])

# 답변 생성
answer_chain = answer_prompt | llm
answer = answer_chain.invoke({"context": context, "question": question})

print("=== 최종 답변 ===")
print(answer.content)

---

## 정리: 검색 관련성 평가

### 핵심 코드

```python
# 1. 평가 스키마
class GradeDocuments(BaseModel):
    binary_score: str = Field(description="'yes' 또는 'no'")

# 2. Structured Output LLM
grader = llm.with_structured_output(GradeDocuments)

# 3. 평가 체인
retrieval_grader = grade_prompt | grader

# 4. 각 문서 평가
for doc in docs:
    result = retrieval_grader.invoke({"question": q, "document": doc})
    if result.binary_score == 'yes':
        relevant_docs.append(doc)
```

### 평가 활용

| 평가 결과 | 다음 행동 |
|----------|----------|
| 관련 문서 있음 | 답변 생성 |
| 관련 문서 없음 | 재검색 또는 웹 검색 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(temperature=0)

# 변경
embeddings = OllamaEmbeddings(model='nomic-embed-text')
llm = ChatOllama(model='llama3.2', temperature=0)
```

## 다음 단계

**LangSmith**를 사용한 체계적인 평가 방법을 배웁니다. (03-06번 노트북)

---
## 코드 10-3~10-6 LangSmith 평가: 에이전트 및 RAG 시스템 평가

# LangSmith 평가: 에이전트 및 RAG 시스템 평가

이 노트북에서는 **LangSmith를 사용한 체계적인 평가 방법**을 개념적으로 배웁니다.

## 왜 체계적인 평가가 필요할까요?

```
┌────────────────────────────────────────────────────────────────────┐
│                    LLM 애플리케이션 평가의 어려움                    │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   전통적인 소프트웨어:                                              │
│   입력 → 함수 → 출력                                               │
│   결과가 결정적 (항상 같은 출력)                                    │
│   테스트: assert output == expected                                │
│                                                                    │
│   LLM 애플리케이션:                                                 │
│   입력 → LLM → 출력                                                │
│   결과가 비결정적 (매번 다른 출력)                                  │
│   "정답"이 명확하지 않음                                           │
│                                                                    │
│   해결: LLM을 평가자로 사용! (LLM-as-Judge)                        │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## LangSmith 평가 아키텍처

```
┌────────────────────────────────────────────────────────────────────┐
│                    LangSmith 평가 흐름                              │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   1️⃣ 데이터셋 생성                                                 │
│   ┌─────────────────────────────────────────┐                      │
│   │ Input (질문)        │ Output (정답)     │                      │
│   ├─────────────────────┼───────────────────┤                      │
│   │ "가장 많이 판매된.." │ "Hot Girl"        │                      │
│   │ "Led Zeppelin..."   │ "14개의 앨범"     │                      │
│   └─────────────────────┴───────────────────┘                      │
│                                                                    │
│   2️⃣ 예측 함수 정의                                                │
│   def predict(example):                                            │
│       return agent.invoke(example["input"])                        │
│                                                                    │
│   3️⃣ 평가자 정의                                                   │
│   def evaluator(run, example):                                     │
│       return {"score": compare(run.output, example.output)}        │
│                                                                    │
│   4️⃣ 평가 실행                                                     │
│   evaluate(predict, data=dataset, evaluators=[evaluator])          │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 평가 유형

## 1.1 답변 정확도 평가

In [ ]:
# 의사 코드 - LangSmith 연동 필요

# 답변 정확도 평가자
def answer_evaluator(run, example):
    """
    RAG 답변 정확도 평가
    
    LLM을 사용하여 정답과 예측 답변 비교
    """
    # 입력, 정답, 예측값
    input_question = example.inputs["input"]
    reference = example.outputs["output"]  # 정답
    prediction = run.outputs["response"]   # 모델 응답
    
    # LLM 평가자
    llm = ChatOllama(model="llama3.2", temperature=0)
    
    # 평가 프롬프트
    eval_prompt = f"""
    질문: {input_question}
    정답: {reference}
    학생 답변: {prediction}
    
    학생의 답변이 정답과 의미적으로 일치하면 1, 아니면 0을 반환하세요.
    """
    
    score = llm.invoke(eval_prompt)
    
    return {"key": "answer_accuracy", "score": int(score)}

print("✅ 답변 정확도 평가자 개념 설명 완료")

## 1.2 도구 호출 평가

In [ ]:
# 의사 코드 - 도구 호출 평가

def check_specific_tool_call(run, example):
    """
    첫 번째 도구 호출이 예상대로인지 확인
    """
    expected_tool = 'sql_db_list_tables'
    
    response = run.outputs["response"]
    
    # 도구 호출 가져오기
    try:
        tool_call = response.tool_calls[0]['name']
    except:
        tool_call = None
    
    score = 1 if tool_call == expected_tool else 0
    
    return {"key": "first_tool_call", "score": score}

print("✅ 도구 호출 평가자 개념 설명 완료")

## 1.3 에이전트 경로(Trajectory) 평가

In [ ]:
# 의사 코드 - 에이전트 경로 평가

def contains_all_tool_calls_in_order(run, example):
    """
    에이전트가 예상된 도구들을 올바른 순서로 호출했는지 확인
    """
    expected = [
        'sql_db_list_tables',  # 1. 테이블 목록 조회
        'sql_db_schema',       # 2. 스키마 조회
        'sql_db_query_checker', # 3. 쿼리 검증
        'sql_db_query',        # 4. 쿼리 실행
        'check_result'         # 5. 결과 확인
    ]
    
    messages = run.outputs["response"]
    
    # 실제 도구 호출 추출
    tool_calls = [
        tc['name'] for m in messages 
        for tc in getattr(m, 'tool_calls', [])
    ]
    
    # 순서대로 포함되어 있는지 확인
    it = iter(tool_calls)
    score = 1 if all(elem in it for elem in expected) else 0
    
    return {"key": "trajectory_in_order", "score": score}

print("✅ 에이전트 경로 평가자 개념 설명 완료")

# 2. 평가 데이터셋 예시

In [ ]:
# SQL 에이전트 평가용 데이터셋 예시

sql_evaluation_examples = [
    {
        "input": "어느 나라의 고객이 가장 많이 지출했나요? 그리고 얼마를 지출했나요?",
        "output": "가장 많이 지출한 나라는 미국으로, 총 지출액은 $523.06입니다"
    },
    {
        "input": "2013년에 가장 많이 판매된 트랙은 무엇인가요?",
        "output": "2013년에 가장 많이 판매된 트랙은 Hot Girl입니다."
    },
    {
        "input": "Led Zeppelin 아티스트는 몇 개의 앨범을 발매했나요?",
        "output": "Led Zeppelin은 14개의 앨범을 발매했습니다"
    },
    {
        "input": "'Big Ones' 앨범의 총 가격은 얼마인가요?",
        "output": "'Big Ones' 앨범의 총 가격은 14.85입니다"
    },
    {
        "input": "2009년에 어떤 영업 담당자가 가장 많은 매출을 올렸나요?",
        "output": "Steve Johnson이 2009년에 가장 많은 매출을 올렸습니다"
    },
]

print("=== SQL 에이전트 평가 데이터셋 ===")
for i, ex in enumerate(sql_evaluation_examples):
    print(f"\n[예제 {i+1}]")
    print(f"  질문: {ex['input']}")
    print(f"  정답: {ex['output']}")

# 3. 평가 실행 (개념)

```python
from langsmith import Client
from langsmith.evaluation import evaluate

# 1. LangSmith 클라이언트 생성
client = Client()

# 2. 데이터셋 생성
dataset = client.create_dataset("sql-agent-eval")
client.create_examples(
    inputs=[{"input": ex["input"]} for ex in examples],
    outputs=[{"output": ex["output"]} for ex in examples],
    dataset_id=dataset.id
)

# 3. 예측 함수
def predict(example):
    result = agent.invoke({"messages": [("user", example["input"])]})
    return {"response": result["messages"][-1].content}

# 4. 평가 실행
results = evaluate(
    predict,
    data="sql-agent-eval",
    evaluators=[answer_evaluator, trajectory_evaluator],
    num_repetitions=3,  # 3번 반복 실행
    experiment_prefix="sql-agent-v1"
)
```

---

## 정리: LangSmith 평가

### 평가 유형

| 평가 유형 | 측정 대상 | 평가 방법 |
|----------|----------|----------|
| **답변 정확도** | 최종 응답 | LLM-as-Judge |
| **도구 호출** | 올바른 도구 선택 | 직접 비교 |
| **에이전트 경로** | 도구 호출 순서 | 순서 비교 |

### 핵심 API

```python
# LangSmith 평가
from langsmith.evaluation import evaluate

results = evaluate(
    predict_fn,           # 예측 함수
    data=dataset_name,    # 데이터셋
    evaluators=[...],     # 평가자 목록
    num_repetitions=3,    # 반복 횟수
)
```

### 평가자 구조

```python
def evaluator(run, example) -> dict:
    # run.outputs: 모델 출력
    # example.inputs: 입력
    # example.outputs: 정답
    
    score = compute_score(...)
    
    return {
        "key": "metric_name",
        "score": score  # 0 또는 1
    }
```

## ch10 요약: 평가

| 기능 | 용도 |
|------|------|
| **검색 관련성 평가** | RAG 문서 필터링 |
| **답변 정확도** | 최종 응답 품질 |
| **도구 호출 평가** | 에이전트 동작 검증 |
| **경로 평가** | 에이전트 추론 과정 검증 |

## 전체 커리큘럼 요약

| 챕터 | 주제 |
|------|------|
| ch01 | LangChain 기초, 프롬프트, 체인 |
| ch02 | RAG 기초, 벡터 DB |
| ch03 | 고급 RAG, Agentic RAG |
| ch04 | 메모리 관리 |
| ch05 | LangGraph 기본 챗봇 |
| ch06 | 에이전트와 도구 |
| ch07 | 고급 패턴 (Reflection, Subgraph, Supervisor) |
| ch08 | 고급 기능 (Streaming, Interrupt, State) |
| ch09 | 배포 |
| ch10 | 평가 |